# Modèles et prédictions

#### 1. Préparation des données (X, y, split)

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

from sklearn.linear_model import LogisticRegression, Lasso, ElasticNet, BayesianRidge
import xgboost as xgb

# Exemple si ton DataFrame s'appelle df et que la target est 'RET'
# (True/False -> classification binaire)
X = train.drop(columns=['RET'])
y = train['RET'].astype(int)  # 0/1

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


#### 2. Fonction utilitaire pour évaluer les modèles

In [ ]:
def eval_model(name, model, X_train, y_train, X_test, y_test):
    """
    Affiche Accuracy, F1 et ROC-AUC sur train et test.
    """
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    # certaines méthodes comme XGBoost peuvent renvoyer proba ou logits
    if hasattr(model, "predict_proba"):
        y_proba_test = model.predict_proba(X_test)[:, 1]
    elif hasattr(model, "decision_function"):
        y_proba_test = model.decision_function(X_test)
    else:
        # fallback: proba approchée par la prédiction binaire
        y_proba_test = y_pred_test

    print(f"====== {name} ======")
    print("Train accuracy :", accuracy_score(y_train, y_pred_train))
    print("Test  accuracy :", accuracy_score(y_test, y_pred_test))
    print("Train F1       :", f1_score(y_train, y_pred_train))
    print("Test  F1       :", f1_score(y_test, y_pred_test))
    print("Test  ROC-AUC  :", roc_auc_score(y_test, y_proba_test))
    print()


#### 3. Logistic Regression (baseline + GridSearch)

In [ ]:
logit_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),  # with_mean=False si sparse
    ("clf", LogisticRegression(max_iter=500, n_jobs=-1))
])

param_logit = {
    "clf__C": [0.01, 0.1, 1, 10],
    "clf__penalty": ["l2"],
    "clf__solver": ["lbfgs", "saga"]
}

grid_logit = GridSearchCV(
    logit_pipe,
    param_grid=param_logit,
    cv=3,
    scoring="f1",
    n_jobs=-1,
    verbose=1
)

grid_logit.fit(X_train, y_train)
print("Best params Logistic:", grid_logit.best_params_)
print("Best CV score (F1)  :", grid_logit.best_score_)

best_logit = grid_logit.best_estimator_
eval_model("Logistic Regression (GridSearch)", best_logit, X_train, y_train, X_test, y_test)


#### 4. Lasso (classification via régression logistique L1)

In [ ]:
lasso_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("clf", LogisticRegression(
        penalty="l1",
        solver="saga",
        max_iter=1000,
        n_jobs=-1
    ))
])

param_lasso = {
    "clf__C": [0.01, 0.1, 1, 10],
}

grid_lasso = GridSearchCV(
    lasso_pipe,
    param_grid=param_lasso,
    cv=3,
    scoring="f1",
    n_jobs=-1,
    verbose=1
)

grid_lasso.fit(X_train, y_train)
print("Best params Lasso (Logit L1):", grid_lasso.best_params_)
print("Best CV score (F1)         :", grid_lasso.best_score_)

best_lasso = grid_lasso.best_estimator_
eval_model("Lasso Logistic (L1)", best_lasso, X_train, y_train, X_test, y_test)


Le modèle Lasso logistique reprend la même idée que la régression logistique mais ajoute une pénalité L1 sur les coefficients, ce qui force certains d’entre eux à devenir exactement nuls.
Concrètement, il sert à faire de la sélection automatique de variables en conservant surtout les signaux les plus pertinents dans les features, et le GridSearch fait varier le paramètre de régularisation pour afficher les performances (Accuracy, F1, ROC‑AUC) du meilleur compromis entre complexité du modèle et qualité de prédiction.

#### 5. Bridge / ElasticNet (L1 + L2)

In [ ]:
enet_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("clf", LogisticRegression(
        penalty="elasticnet",
        solver="saga",
        max_iter=1000,
        n_jobs=-1
    ))
])

param_enet = {
    "clf__C": [0.01, 0.1, 1, 10],
    "clf__l1_ratio": [0.1, 0.5, 0.9]  # 0.1 ~ plus L2, 0.9 ~ plus L1
}

grid_enet = GridSearchCV(
    enet_pipe,
    param_grid=param_enet,
    cv=3,
    scoring="f1",
    n_jobs=-1,
    verbose=1
)

grid_enet.fit(X_train, y_train)
print("Best params ElasticNet:", grid_enet.best_params_)
print("Best CV score (F1)    :", grid_enet.best_score_)

best_enet = grid_enet.best_estimator_
eval_model("ElasticNet Logistic", best_enet, X_train, y_train, X_test, y_test)


Le modèle ElasticNet (Bridge dans contexte) combine une pénalité L1 et une pénalité L2 afin de bénéficier à la fois de la sélection de variables (comme Lasso) et de la stabilisation des coefficients (comme Ridge).
Il sert à contrôler plus finement la régularisation via le paramètre l1_ratio, et le GridSearch teste plusieurs niveaux de régularisation pour afficher le jeu de paramètres qui maximise le F1‑score tout en gardant un modèle robuste dans un environnement financier potentiellement bruité.

#### 6. Bayesian Ridge (version classification simple)

In [ ]:
from sklearn.linear_model import BayesianRidge
from sklearn.base import BaseEstimator, ClassifierMixin

class BayesianRidgeClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, **kwargs):
        self.model = BayesianRidge(**kwargs)

    def fit(self, X, y):
        # y en 0/1
        self.model.fit(X, y)
        return self

    def predict_proba(self, X):
        # approx: output = mean prediction, variance ignorée
        y_hat = self.model.predict(X)
        # clip pour rester dans [0,1]
        y_hat = np.clip(y_hat, 0, 1)
        return np.vstack([1 - y_hat, y_hat]).T

    def predict(self, X):
        proba = self.predict_proba(X)[:, 1]
        return (proba >= 0.5).astype(int)

bayes_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("clf", BayesianRidgeClassifier())
])

param_bayes = {
    # paramètres éventuels de BayesianRidge si tu veux tuner
    "clf__model__alpha_1": [1e-6, 1e-4],
    "clf__model__alpha_2": [1e-6, 1e-4],
    "clf__model__lambda_1": [1e-6, 1e-4],
    "clf__model__lambda_2": [1e-6, 1e-4],
}

grid_bayes = GridSearchCV(
    bayes_pipe,
    param_grid=param_bayes,
    cv=3,
    scoring="f1",
    n_jobs=-1,
    verbose=1
)

grid_bayes.fit(X_train, y_train)
print("Best params BayesianRidge:", grid_bayes.best_params_)
print("Best CV score (F1)      :", grid_bayes.best_score_)

best_bayes = grid_bayes.best_estimator_
eval_model("Bayesian Ridge (Classifier wrapper)", best_bayes, X_train, y_train, X_test, y_test)


Le modèle bayésien utilisé repose sur une régression de type Bayesian Ridge adaptée en classifieur pour une cible binaire, ce qui signifie qu’au lieu de produire un seul vecteur de coefficients, il modélise une distribution a priori et a posteriori sur ces coefficients.
Il sert surtout à quantifier l’incertitude sur les paramètres du modèle et donc implicitement sur les prédictions, et le GridSearch fait varier les hyperparamètres de la prior (par exemple alpha_1, lambda_1) tout en affichant les performances de classification finales (Accuracy, F1, ROC‑AUC) après conversion en probabilités.

#### 7. XGBoost (sans ensembles scikit‑learn)

In [ ]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    tree_method="hist",
    n_estimators=300,
    n_jobs=-1,
    random_state=42
)

param_xgb = {
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "gamma": [0, 1]
}

grid_xgb = GridSearchCV(
    xgb_clf,
    param_grid=param_xgb,
    cv=3,
    scoring="f1",
    n_jobs=-1,
    verbose=1
)

grid_xgb.fit(X_train, y_train)
print("Best params XGBoost:", grid_xgb.best_params_)
print("Best CV score (F1):", grid_xgb.best_score_)

best_xgb = grid_xgb.best_estimator_
eval_model("XGBoost", best_xgb, X_train, y_train, X_test, y_test)


XGBoost est un modèle non linéaire basé sur un ensemble séquentiel d’arbres de décision où chaque nouvel arbre corrige les erreurs du précédent, ce qui le rend très puissant pour capturer des relations complexes et des interactions entre les variables.​
Dans ton projet, il sert à explorer des structures de dépendance plus riches entre les retours/volumes historiques et le signe du retour futur, et le GridSearch fait varier des hyperparamètres clés (profondeur des arbres, learning rate, subsample, colsample, gamma) pour afficher les meilleurs scores de classification sur les données de validation et de test.